<a href="https://colab.research.google.com/github/guilhermeSuporteDev/colab/blob/main/zero_shot_portuguese_product_category_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [1]:
!pip install transformers
!pip install datasets
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install torch
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.4 MB/s eta 0:00:00


# Build Environment

In [6]:
import os

def create_folder(folder_name):
  if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    log(f"Diretório '{folder_name}' criado com sucesso.", green_color)
  else:
    log(f"Diretório '{folder_name}' já existe.")

create_folder("results")

Diretório 'results' criado com sucesso.


# Global Definitions

In [20]:
import os
def get_last_trained_model_name(path, default_model):
    entries = os.listdir(path)
    directories = [entry for entry in entries if os.path.isdir(os.path.join(path, entry))]

    # Filtra e ordena os diretórios que seguem o formato "checkpoint-XX" e que XX é um número
    valid_directories = [dir for dir in directories if '-' in dir and dir.split('-')[1].isdigit()]
    valid_directories.sort(key=lambda x: int(x.split('-')[1]))

    model_name = valid_directories[-1] if valid_directories else default_model
    return model_name

In [21]:
# Definir códigos ANSI para cores
red_color = "\033[91m"
green_color = "\033[92m"
yellow_color = "\033[93m"
blue_color = "\033[0;34m"
reset_color = "\033[0m"

def log(text, color = blue_color):
  print(f"{color}{text}{reset_color}")

In [22]:
# Definições de validação
def verificar_categorias(produtos, categorias):
    categorias_invalidas = set()
    for produto in produtos:
        if produto[1] not in categorias:
            categorias_invalidas.add(produto[1])

    if categorias_invalidas:
        print(f"Categorias inválidas encontradas: {categorias_invalidas}", red_color)
    else:
        log("Todas as categorias são válidas.", green_color)

def verificar_duplicatas(produtos):
    visto = set()
    duplicatas = set()
    for produto in produtos:
        if produto[0] in visto:
            duplicatas.add(produto[0])
        visto.add(produto[0])

    if duplicatas:
        log(f"Duplicatas encontradas: {duplicatas}" , red_color)
    else:
        log("Nenhuma duplicata encontrada.", green_color)

def verificar_itens_nao_categorizados(produtos):
    itens_nao_categorizados = [produto for produto in produtos if produto[1] == ""]

    if itens_nao_categorizados:
        log(f"Itens não categorizados encontrados: {itens_nao_categorizados}", red_color)
    else:
        log("Todos os itens estão categorizados.", green_color)

def validar_rotulos_de_classe(produtos, categorias):
    # Primeiro, mapear as categorias para números inteiros
    categoria_para_inteiro = {categoria: i for i, categoria in enumerate(categorias)}

    # Verificar se todos os produtos têm rótulos válidos
    rotulos_invalidos = []
    for produto in produtos:
        categoria = produto[1]
        if categoria not in categoria_para_inteiro:
            rotulos_invalidos.append((produto[0], categoria))

    # Exibir um relatório
    if rotulos_invalidos:
        log("Foram encontrados rótulos de classe inválidos:", red_color)
        for item in rotulos_invalidos:
            print(f"Produto: {item[0]}, Categoria: {item[1]}")
    else:
        log("Todos os rótulos de classe são válidos.", green_color)


# Global Vars

In [59]:
# CONSTS
__EPOCHS__ = 10
__TRAINING_MODEL__ = "facebook/bart-large-mnli"
__RESULTS_PATH__ = "./results"
__CURRENT_TRAINING_MODEL__ = get_last_trained_model_name(__RESULTS_PATH__, __TRAINING_MODEL__)
__LAST_MODEL_ACCURACY__ = "0.00%"

log(f"INICIANDO TREINAMENTO DO MODELO: {blue_color}{__CURRENT_TRAINING_MODEL__}", green_color)

INICIANDO TREINAMENTO DO MODELO: checkpoint-155


# Dataset

In [24]:
produtos = [
    # Higiene
    ["DESODORANTE ANTITRANSPIRANTE ACE ROLL-ON", "higiene"],
    ["SABONETE DOVE ORIGINAL", "higiene"],
    ["SHAMPOO ANTICASPA", "higiene"],
    ["PAPEL HIGIÊNICO", "higiene"],
    ["COTONETE 75 UNIDADES", "higiene"],
    ["LENÇO DE PAPEL 10 UNIDADES", "higiene"],
    ["ABSORVENTE NOTURNO 8 UNIDADES", "higiene"],
    ["ESCOVA DE DENTE MACIA", "higiene"],
    ["GEL DE BARBEAR 100G", "higiene"],
    ["ÁLCOOL GEL 70% 500ML", "higiene"],
    ["CONDICIONADOR CABELOS SECOS 200ML", "higiene"],
    ["ANTISSÉPTICO BUCAL 250ML", "higiene"],
    ["SABONETE LÍQUIDO ÍNTIMO 200ML", "higiene"],
    ["FRALDA GERIÁTRICA G 8 UNIDADES", "higiene"],
    ["FIO DENTAL 50M", "higiene"],
    ["SABONETE DE GLICERINA", "higiene"],
    ["DESODORANTE SPRAY 150ML", "higiene"],
    ["SHAMPOO ANTIQUEDA 200ML", "higiene"],
    ["CONDICIONADOR REPARADOR 200ML", "higiene"],
    ["ESPUMA DE BARBEAR 200ML", "higiene"],
    ["CREME DE BARBEAR 100G", "higiene"],
    ["CREME PARA MÃOS 75ML", "higiene"],
    ["CREME FACIAL HIDRATANTE 50G", "higiene"],
    ["PROTETOR SOLAR FPS 50 100ML", "higiene"],
    ["CREME PARA AS MÃOS 100ML", "higiene"],
    ["SHAMPOO PARA CABELOS OLEOSOS 200ML", "higiene"],
    ["CONDICIONADOR PARA CABELOS CACHEADOS 200ML", "higiene"],
    ["SABONETE ESFOLIANTE 200G", "higiene"],
    ["CREME DENTAL INFANTIL 50G", "higiene"],
    ["DESODORANTE NATURAL 50ML", "higiene"],
    ["LOÇÃO HIDRATANTE 200ML", "higiene"],
    ["ÁLCOOL EM GEL 70% 100ML", "higiene"],
    ["SHAMPOO INFANTIL 200ML", "higiene"],
    ["ESPONJA DE BANHO", "higiene"],
    ["ÁGUA OXIGENADA 100ML", "higiene"],
    ["CREME DE BARBEAR PELE SENSÍVEL 100G", "higiene"],
    ["ESPUMA DE LIMPEZA FACIAL 150ML", "higiene"],
    ["FIXADOR DE DENTADURA 40G", "higiene"],
    ["GEL ANTISSÉPTICO 70% 250ML", "higiene"],
    ["HIDRATANTE LABIAL 10G", "higiene"],
    ["LOÇÃO PÓS DEPILAÇÃO 200ML", "higiene"],
    ["MÁSCARA FACIAL DE ARGILA 100G", "higiene"],
    ["ÓLEO PARA BARBA 30ML", "higiene"],
    ["PAPEL HIGIÊNICO FOLHA DUPLA 4 ROLOS", "higiene"],
    ["PERFUME MASCULINO 50ML", "higiene"],
    ["PINÇA PARA SOBRANCELHAS", "higiene"],
    ["REMOVEDOR DE MAQUIAGEM 100ML", "higiene"],
    ["SABONETE ANTISSEPTICO 90G", "higiene"],
    ["SABONETE ÍNTIMO FEMININO 200ML", "higiene"],
    ["SHAMPOO A SECO 150ML", "higiene"],
    ["TALCO ANTISSEPTICO 100G", "higiene"],
    ["TOALHAS UMEDECIDAS 50 UNIDADES", "higiene"],
    ["ESCOVA DE LIMPEZA FACIAL", "higiene"],
    ["CREME PARA PÉS 100ML", "higiene"],


    # Laticínios
    ["LEITE INTEGRAL UHT", "laticínios"],
    ["QUEIJO MUSSARELA FATIADO", "laticínios"],
    ["CREME DE LEITE FRESCO", "laticínios"],
    ["IOGURTE NATURAL INTEGRAL", "laticínios"],
    ["QUEIJO RICOTA FRESCA", "laticínios"],
    ["LEITE FERMENTADO 100G", "laticínios"],
    ["YOGURTE GREGO 100G", "laticínios"],
    ["COALHADA SECA 200G", "laticínios"],
    ["LEITE EM PÓ INTEGRAL 400G", "laticínios"],
    ["QUEIJO GOUDA 100G", "laticínios"],
    ["CREME DE RICOTA 150G", "laticínios"],
    ["LEITE CONDENSADO 395G", "laticínios"],
    ["DOCE DE LEITE 400G", "laticínios"],
    ["QUEIJO PROVOLONE 100G", "laticínios"],
    ["QUEIJO BRIE 125G", "laticínios"],
    ["LEITE DE CABRA 1L", "laticínios"],
    ["QUEIJO COTTAGE 200G", "laticínios"],
    ["QUEIJO PARMESÃO RALADO 100G", "laticínios"],
    ["IOGURTE DE COCO 170G", "laticínios"],
    ["SORVETE DE IOGURTE 1L", "laticínios"],
    ["LEITE SEM LACTOSE 1L", "laticínios"],
    ["QUEIJO FETA 200G", "laticínios"],
    ["MANTEIGA GHEE 200G", "laticínios"],
    ["QUEIJO CAMEMBERT 125G", "laticínios"],
    ["LEITE VEGETAL DE AVEIA 1L", "laticínios"],
    ["QUEIJO ROQUEFORT 100G", "laticínios"],
    ["LEITE FLAN 200G", "laticínios"],
    ["YOGURTE DE AMORA 170G", "laticínios"],
    ["CREME DE LEITE NATA 300G", "laticínios"],
    ["LEITE DE ARROZ 1L", "laticínios"],
    ["QUEIJO EDAM 150G", "laticínios"],
    ["LEITE DE COCO PARA BEBER 1L", "laticínios"],
    ["IOGURTE DE BANANA 170G", "laticínios"],
    ["LEITE DE SOJA 1L", "laticínios"],
    ["QUEIJO QUARK 150G", "laticínios"],
    ["QUEIJO MINAS CURADO 200G", "laticínios"],
    ["LEITE DE AMÊNDOA 1L", "laticínios"],
    ["QUEIJO PROVOLONE 200G", "laticínios"],
    ["LEITE DE AVELÃ 1L", "laticínios"],
    ["LEITE DE MACADÂMIA 1L", "laticínios"],
    ["LEITE DESNATADO 1L", "laticínios"],
    ["SORVETE DE FLOCOS 2L", "laticínios"],
    ["IOGURTE DE FRAMBOESA 170G", "laticínios"],
    ["QUEIJO MINAS FRESCAL LIGHT 200G", "laticínios"],
    ["CREME DE RICOTA LIGHT 150G", "laticínios"],
    ["IOGURTE DE MEL 170G", "laticínios"],
    ["QUEIJO CANASTRA 200G", "laticínios"],
    ["IOGURTE DE MANGA 170G", "laticínios"],
    ["LEITE CONDENSADO DIET 395G", "laticínios"],
    ["IOGURTE COM PEDAÇOS DE FRUTA 170G", "laticínios"],
    ["IOGURTE GREGO LIGHT 100G", "laticínios"],
    ["QUEIJO PRATO LIGHT 150G", "laticínios"],
    ["SORVETE DE ABACAXI 1L", "laticínios"],
    ["LEITE VEGETAL DE NOZ PECÃ 1L", "laticínios"],
    ["LEITE VEGETAL DE CASTANHA DE CAJU 1L", "laticínios"],

    # Doces
    ["BISCOITO RECHEADO CHOCOLATE", "doces"],
    ["CHOCOLATE AO LEITE 100G", "doces"],
    ["GELEIA DE MORANGO DIET", "doces"],
    ["BOLACHA ÁGUA E SAL", "doces"],
    ["BALA DE COCO 100G", "doces"],
    ["PIRULITO SABOR FRUTAS", "doces"],
    ["BARRA DE CEREAL CHOCOLATE", "doces"],
    ["GOMA DE MASCAR MENTA", "doces"],
    ["BOMBOM SORTIDO 200G", "doces"],
    ["PUDIM DE CHOCOLATE", "doces"],
    ["WAFER DE MORANGO", "doces"],
    ["GELATINA DE UVA", "doces"],
    ["BRIGADEIRO POTE 200G", "doces"],
    ["PÃO DE MEL INDIVIDUAL", "doces"],
    ["PÉ DE MOLEQUE 100G", "doces"],
    ["PAÇOCA ROLHA", "doces"],
    ["QUINDIM INDIVIDUAL", "doces"],
    ["BARRA DE CHOCOLATE AMARGO 100G", "doces"],
    ["SUSPIRO 100G", "doces"],
    ["MILKSHAKE EM PÓ 200G", "doces"],
    ["CHOCOTONE 500G", "doces"],
    ["GOIABADA CASCÃO 300G", "doces"],
    ["TORRONE 100G", "doces"],
    ["BANANA PASSA 100G", "doces"],
    ["BARRA DE CEREAL DE FRUTAS", "doces"],
    ["BOLO DE CHOCOLATE 100G", "doces"],
    ["CHURROS CONGELADO", "doces"],
    ["DOCE DE COCO 200G", "doces"],
    ["GELATINA DE LIMÃO", "doces"],
    ["MARSHMALLOW 100G", "doces"],
    ["PASTEL DE NATA", "doces"],
    ["TORTA DE LIMÃO 300G", "doces"],
    ["TRUFA DE CHOCOLATE 100G", "doces"],
    ["BISCOITO DE MAIZENA 150G", "doces"],
    ["BARRA DE CHOCOLATE COM AMENDOAS 100G", "doces"],
    ["BISCOITO WAFFLE 100G", "doces"],
    ["BALAS DE GENGIBRE 100G", "doces"],
    ["BALAS DE MENTA 100G", "doces"],
    ["BOMBONS FINOS SORTIDOS 200G", "doces"],
    ["CHOCOLATE QUENTE EM PÓ 200G", "doces"],
    ["DOCE DE ABÓBORA 300G", "doces"],
    ["DOCE DE BANANA 300G", "doces"],
    ["GELEIA DE MARMELADA 200G", "doces"],
    ["CHOCOLATE AMARGO 85% CACAU 100G", "doces"],
    ["PÉ DE MOLEQUE 200G", "doces"],
    ["BARRA DE CEREAL DE MEL E NUTS", "doces"],
    ["BISCOITO DE GENGIBRE 150G", "doces"],
    ["TORRONE DE AMENDOIM 100G", "doces"],
    ["BOMBOM DE LICOR 200G", "doces"],
    ["PASTILHAS DE CHOCOLATE 100G", "doces"],
    ["BALA DE CARAMELO 100G", "doces"],
    ["BISCOITO DE CHOCOLATE BRANCO 150G", "doces"],
    ["COCADA BRANCA 200G", "doces"],

    # Carnes
    ["PEITO DE FRANGO CONGELADO", "carnes"],
    ["FILE DE TILAPIA CONGELADO", "carnes"],
    ["ALCATRA COMPLETA KG", "carnes"],
    ["COSTELA SUINA CONGELADA", "carnes"],
    ["LINGUIÇA TOSCANA", "carnes"],
    ["COSTELA BOVINA KG", "carnes"],
    ["PEIXE SALMÃO FILE 200G", "carnes"],
    ["FRANGO A PASSARINHO KG", "carnes"],
    ["CARNE SUÍNA MOÍDA KG", "carnes"],
    ["LINGUIÇA CALABRESA 200G", "carnes"],
    ["HAMBÚRGUER SUÍNO 4 UNIDADES", "carnes"],
    ["ALMONDEGA BOVINA 300G", "carnes"],
    ["BISTECA SUÍNA KG", "carnes"],
    ["COXA DE FRANGO KG", "carnes"],
    ["SALSICHA FRANGO 500G", "carnes"],
    ["BACON EM FATIAS 200G", "carnes"],
    ["COSTELA DE CORDEIRO KG", "carnes"],
    ["PERNIL SUÍNO KG", "carnes"],
    ["RABADA BOVINA KG", "carnes"],
    ["CARNE DE JAVALI KG", "carnes"],
    ["FÍGADO BOVINO KG", "carnes"],
    ["CARNE DE PATO KG", "carnes"],
    ["MIOLO DE ALCATRA KG", "carnes"],
    ["CORDEIRO MOÍDO KG", "carnes"],
    ["FILE MIGNON SUÍNO KG", "carnes"],
    ["CARNE DE BÚFALO KG", "carnes"],
    ["FRANGO ORGÂNICO KG", "carnes"],
    ["CARNE DE CORDEIRO KG", "carnes"],
    ["COSTELA DE PORCO KG", "carnes"],
    ["PEIXE DOURADO KG", "carnes"],
    ["COSTELA DE BOI KG", "carnes"],
    ["CARNE DE COELHO KG", "carnes"],
    ["PERDIZ KG", "carnes"],
    ["LINGUIÇA DE PERNIL 500G", "carnes"],
    ["COSTELA DE VACA KG", "carnes"],
    ["FRANGO CAIPIRA KG", "carnes"],
    ["CARNE DE PORCO ALCATRA KG", "carnes"],
    ["BIFE DE CHORIZO KG", "carnes"],
    ["PEITO DE PERU KG", "carnes"],
    ["CARNE DE VEADO KG", "carnes"],
    ["CARNE DE CABRA KG", "carnes"],
    ["PATO INTEIRO KG", "carnes"],
    ["PERU INTEIRO KG", "carnes"],
    ["COXA DE FRANGO COM PELE KG", "carnes"],
    ["SALSICHA DE PERU 500G", "carnes"],
    ["CARNE DE FÍGADO BOVINO KG", "carnes"],
    ["MÚSCULO BOVINO KG", "carnes"],
    ["CARNE MOÍDA DE PERU 500G", "carnes"],
    ["BIFE DE FIGADO KG", "carnes"],
    ["CARNE DE JAVALI SELVAGEM KG", "carnes"],
    ["FRANGO ORGÂNICO DESFIADO 500G", "carnes"],
    ["PEITO DE PATO KG", "carnes"],
    ["COSTELA SUÍNA DEFUMADA KG", "carnes"],
    ["CARNE SECA DESFIADA 500G", "carnes"],

    # Grãos e Cereais
    ["CEREAL SUCRILHOS KELLOGS 700G", "grãos e cereais"],
    ["ARROZ AGULHINHA TIPO 1", "grãos e cereais"],
    ["FARINHA DE TRIGO INTEGRAL", "grãos e cereais"],
    ["AVEIA EM FLOCOS 200G", "grãos e cereais"],
    ["SEMENTE DE CHIA 100G", "grãos e cereais"],
    ["FARINHA DE AVEIA 500G", "grãos e cereais"],
    ["FAROFA PRONTA 250G", "grãos e cereais"],
    ["ARROZ INTEGRAL 1KG", "grãos e cereais"],
    ["TRIGO PARA QUIBE 500G", "grãos e cereais"],
    ["FARINHA DE MANDIOCA 500G", "grãos e cereais"],
    ["FLOCOS DE MILHO SEM AÇÚCAR", "grãos e cereais"],
    ["LENTILHA 500G", "grãos e cereais"],
    ["ERVILHA SECA 500G", "grãos e cereais"],
    ["SEMENTE DE LINHAÇA 200G", "grãos e cereais"],
    ["FARINHA DE CENTEIO 500G", "grãos e cereais"],
    ["FARINHA DE COCO 200G", "grãos e cereais"],
    ["MIX DE NUTS 100G", "grãos e cereais"],
    ["FARINHA DE QUINOA 500G", "grãos e cereais"],
    ["TRIGO SARRACENO 500G", "grãos e cereais"],
    ["CEVADA EM GRÃO 500G", "grãos e cereais"],
    ["SEMENTE DE ABÓBORA 100G", "grãos e cereais"],
    ["AMÊNDOAS 100G", "grãos e cereais"],
    ["CASTANHA-DO-PARÁ 100G", "grãos e cereais"],
    ["FARINHA DE MILHO 1KG", "grãos e cereais"],
    ["CEVADA 500G", "grãos e cereais"],
    ["FARINHA DE CHIA 200G", "grãos e cereais"],
    ["TRIGO PARA KIBE 500G", "grãos e cereais"],
    ["FARINHA DE MANDIOCA 1KG", "grãos e cereais"],
    ["ARROZ CARIOCA 1KG", "grãos e cereais"],
    ["FARINHA DE GRÃO DE BICO 500G", "grãos e cereais"],
    ["MILHO DE PIPOCA 500G", "grãos e cereais"],
    ["AVEIA EM FLOCOS 500G", "grãos e cereais"],
    ["FARINHA DE TAPIOCA 500G", "grãos e cereais"],
    ["SEMENTE DE GIRASSOL 200G", "grãos e cereais"],
    ["FARINHA DE ARROZ INTEGRAL 1KG", "grãos e cereais"],
    ["ARROZ SELVAGEM 500G", "grãos e cereais"],
    ["FARINHA DE SORGO 500G", "grãos e cereais"],
    ["FARINHA DE MILHO AMARELO 1KG", "grãos e cereais"],
    ["FARINHA DE BATATA DOCE 500G", "grãos e cereais"],
    ["AVEIA EM FLOCOS FINOS 500G", "grãos e cereais"],
    ["FARINHA DE TRIGO PARA PÃO 1KG", "grãos e cereais"],
    ["QUINOA BRANCA EM GRÃO 500G", "grãos e cereais"],
    ["FARINHA DE TRIGO SARRACENO 500G", "grãos e cereais"],
    ["GRÃO DE BICO EM CONSERVA", "grãos e cereais"],
    ["FARINHA DE LINHAÇA DOURADA 500G", "grãos e cereais"],
    ["ARROZ INTEGRAL PRETO 1KG", "grãos e cereais"],
    ["FARINHA DE BANANA VERDE ORGÂNICA 200G", "grãos e cereais"],
    ["TRIGO EM GRÃO 1KG", "grãos e cereais"],
    ["MILHO VERDE EM CONSERVA", "grãos e cereais"],
    ["ERVILHAS EM CONSERVA", "grãos e cereais"],
    ["FEIJÃO FRADINHO 1KG", "grãos e cereais"],
    ["FEIJÃO PRETO ORGÂNICO 1KG", "grãos e cereais"],

    # Não Perecível
    # ["AZEITE DE OLIVA EXTRA VIRGEM", "não perecível"],
    # ["MACARRÃO COM OVOS", "não perecível"],
    # ["BATATA PALHA", "não perecível"],
    # ["PIMENTA DO REINO MOÍDA", "não perecível"],
    # ["CÁPSULAS DE CAFÉ", "não perecível"],
    # ["FAROFA PRONTA TRADICIONAL", "não perecível"],
    # ["MISTURA PARA BOLO DE CHOCOLATE", "não perecível"],
    # ["SARDINHA EM LATA", "não perecível"],
    # ["MOLHO DE TOMATE 340G", "não perecível"],
    # ["ATUM RALADO EM ÓLEO", "não perecível"],
    # ["SAL REFINADO 1KG", "não perecível"],
    # ["EXTRATO DE TOMATE 140G", "não perecível"],
    # ["ERVILHA EM CONSERVA", "não perecível"],
    # ["PALMITO EM CONSERVA", "não perecível"],
    # ["AZEITONA PRETA 100G", "não perecível"],
    # ["MOLHO SHOYU 150ML", "não perecível"],
    # ["EXTRATO DE SOJA EM PÓ 200G", "não perecível"],
    # ["MOLHO BARBECUE 200ML", "não perecível"],
    # ["MOLHO INGLÊS 150ML", "não perecível"],
    # ["VINAGRE DE MAÇÃ 500ML", "não perecível"],
    # ["KETCHUP 340G", "não perecível"],
    # ["MOSTARDA 200G", "não perecível"],
    # ["TEMPERO PRONTO EM PÓ", "não perecível"],
    # ["CALDO DE GALINHA EM CUBOS", "não perecível"],
    # ["MOLHO DE PIMENTA 150ML", "não perecível"],
    # ["MOLHO TÁRTARO 200ML", "não perecível"],
    # ["VINAGRE DE VINHO 500ML", "não perecível"],
    # ["MOLHO DE MOSTARDA 200ML", "não perecível"],
    # ["MOLHO DE ALHO 150ML", "não perecível"],
    # ["EXTRATO DE TOMATE 350G", "não perecível"],
    # ["CONSERVA DE MILHO 200G", "não perecível"],
    # ["AZEITONAS VERDES 200G", "não perecível"],
    # ["PALMITO EM CONSERVA 300G", "não perecível"],
    # ["PIMENTA BIQUINHO EM CONSERVA 100G", "não perecível"],
    # ["MOLHO DE CHAMPIGNON 200G", "não perecível"],
    # ["GRÃO DE BICO EM LATA", "não perecível"],
    # ["MOLHO DE GENGIBRE 200ML", "não perecível"],
    # ["VINAGRETE 300G", "não perecível"],
    # ["MASSA DE PIZZA PRONTA", "não perecível"],
    # ["CALDO DE LEGUMES EM PÓ", "não perecível"],
    # ["SAL GROSSO IODADO 1KG", "não perecível"],
    # ["ARROZ ARBÓREO 1KG", "não perecível"],
    # ["PIMENTA CALABRESA EM FLOCOS", "não perecível"],
    # ["TEMPERO PARA CARNES", "não perecível"],
    # ["ALHO EM PÓ 100G", "não perecível"],
    # ["CREME DE ASPARGOS EM PÓ", "não perecível"],
    # ["SOPA DE ERVILHA DESIDRATADA", "não perecível"],
    # ["LEITE EM PÓ DESNATADO 200G", "não perecível"],
    # ["SAL ROSA DO HIMALAIA 500G", "não perecível"],
    # ["MISTURA PARA BOLO DE LARANJA", "não perecível"],
    # ["MOLHO PESTO 180G", "não perecível"],
    # ["PURÊ DE BATATA EM FLOCOS", "não perecível"],
    # ["CHÁ MATE SOLÚVEL", "não perecível"],
    # ["BISCOITO ÁGUA E SAL INTEGRAL 200G", "não perecível"],
    # ["PÃO SÍRIO INTEGRAL", "não perecível"],

    # Congelados
    ["SORVETE DE CREME 2L", "congelados"],
    ["ERVILHA CONGELADA", "congelados"],
    ["LASANHA A BOLONHESA PRONTA", "congelados"],
    ["PIZZA CONGELADA 4 QUEIJOS", "congelados"],
    ["FRANGO XADREZ CONGELADO", "congelados"],
    ["LEGUMES CONGELADOS MISTOS", "congelados"],
    ["SORVETE DE CHOCOLATE 2L", "congelados"],
    ["NHOQUE DE BATATA CONGELADO", "congelados"],
    ["FRUTAS CONGELADAS MIX", "congelados"],
    ["COXINHA DA ASA CONGELADA", "congelados"],
    ["PÃO DE QUEIJO MINI 500G", "congelados"],
    ["FRITAS CONGELADAS 1KG", "congelados"],
    ["CAMARÃO CONGELADO LIMPO", "congelados"],
    ["FRANGO À MILANESA CONGELADO", "congelados"],
    ["ESPETINHO DE CARNE CONGELADO", "congelados"],
    ["COXA CREME CONGELADA", "congelados"],
    ["BATATA SMILE CONGELADA", "congelados"],
    ["LEGUMES PARA SOPA CONGELADOS", "congelados"],
    ["EMPADA DE FRANGO CONGELADA", "congelados"],
    ["PIZZA MARGHERITA CONGELADA", "congelados"],
    ["FILE DE MERLUZA CONGELADO", "congelados"],
    ["KIBE CONGELADO", "congelados"],
    ["ACARAJÉ CONGELADO", "congelados"],
    ["FILÉ DE PEIXE BRANCO CONGELADO", "congelados"],
    ["FRANGO EMPANADO CONGELADO", "congelados"],
    ["LEGUMES GRILL CONGELADOS", "congelados"],
    ["SORVETE DE PISTACHE 1L", "congelados"],
    ["HAMBÚRGUER VEGETARIANO CONGELADO", "congelados"],
    ["LASANHA DE QUEIJO CONGELADA", "congelados"],
    ["FRUTAS VERMELHAS CONGELADAS", "congelados"],
    ["ESFIRRA CONGELADA", "congelados"],
    ["PÃO DE ALHO CONGELADO", "congelados"],
    ["CAMARÃO EMPANADO CONGELADO", "congelados"],
    ["STROGONOFF DE FRANGO CONGELADO", "congelados"],
    ["ARROZ DE FORNO CONGELADO", "congelados"],
    ["TORTA DE ESPINAFRE CONGELADA", "congelados"],
    ["SALGADINHO DE QUEIJO CONGELADO", "congelados"],
    ["PIZZA DE ATUM CONGELADA", "congelados"],
    ["SOPA DE LEGUMES CONGELADA", "congelados"],
    ["PUDIM DE CHOCOLATE CONGELADO", "congelados"],
    ["BOLINHO DE BACALHAU CONGELADO", "congelados"],
    ["QUIBE CONGELADO", "congelados"],
    ["SALSICHA CONGELADA", "congelados"],
    ["PÃO INTEGRAL CONGELADO", "congelados"],
    ["FEIJOADA CONGELADA", "congelados"],
    ["COXINHA DE JACA CONGELADA", "congelados"],
    ["BIFE À PARMEGIANA CONGELADO", "congelados"],
    ["TORTA DE LIMÃO CONGELADA", "congelados"],
    ["PEIXE EMPANADO CONGELADO", "congelados"],
    ["RISOTO DE CAMARÃO CONGELADO", "congelados"],
    ["PANQUECA DE CARNE CONGELADA", "congelados"],

    # Padaria
    ["PÃO DE FORMA INTEGRAL", "padaria"],
    ["TORRADA INTEGRAL LIGHT", "padaria"],
    ["PÃO FRANCÊS KG", "padaria"],
    ["CROISSANT DE CHOCOLATE", "padaria"],
    ["BISCOITO DE POLVILHO 100G", "padaria"],
    ["TORTA DE FRANGO KG", "padaria"],
    ["PÃO DE BATATA 100G", "padaria"],
    ["BOLO DE FUBÁ 300G", "padaria"],
    ["PÃO SÍRIO 200G", "padaria"],
    ["ROSCA DOCE 200G", "padaria"],
    ["PÃO DE MEL 100G", "padaria"],
    ["BISCOITO DE NATA 100G", "padaria"],
    ["PÃO DE QUEIJO TRADICIONAL", "padaria"],
    ["MINI SONHO 100G", "padaria"],
    ["BISNAGUINHA 300G", "padaria"],
    ["BROA DE MILHO 200G", "padaria"],
    ["PÃO ITALIANO 500G", "padaria"],
    ["COOKIE INTEGRAL 100G", "padaria"],
    ["BAGEL 100G", "padaria"],
    ["PÃO AUSTRALIANO 200G", "padaria"],
    ["BISCOITO AMANTEIGADO 100G", "padaria"],
    ["ROSCA DE POLVILHO 200G", "padaria"],
    ["PÃO SUECO 200G", "padaria"],
    ["PÃO DE BATATA DOCE 200G", "padaria"],
    ["BAGUETE INTEGRAL 250G", "padaria"],
    ["BISCOITO DE POLVILHO AZEDO 100G", "padaria"],
    ["TORTA DE FRANGO 500G", "padaria"],
    ["PÃO DE QUEIJO 500G", "padaria"],
    ["BISCOITO DE LEITE 200G", "padaria"],
    ["BROWNIE 100G", "padaria"],
    ["BISCOITO DE AVENA 100G", "padaria"],
    ["CROISSANT DE QUEIJO 100G", "padaria"],
    ["PÃO DE ABÓBORA 200G", "padaria"],
    ["PÃO SÍRIO INTEGRAL 200G", "padaria"],
    ["PÃO DE MEL RECHEADO", "padaria"],
    ["ROSCA HÚNGARA 200G", "padaria"],
    ["PÃO DE QUEIJO RECHEADO 300G", "padaria"],
    ["BOLACHA DE CHOCOLATE RECHEADA 150G", "padaria"],
    ["PÃO MULTIGRÃOS 200G", "padaria"],
    ["CROISSANT INTEGRAL 100G", "padaria"],
    ["BISCOITO DE GENGIBRE 100G", "padaria"],
    ["BISCOITO SALGADO 200G", "padaria"],
    ["MUFFIN DE BAUNILHA 100G", "padaria"],
    ["BISCOITO AMANTEIGADO DE COCO 100G", "padaria"],
    ["CUCA ALEMÃ 300G", "padaria"],
    ["BISCOITO DE CHAMPAGNE 150G", "padaria"],
    ["PÃO DE CERVEJA 200G", "padaria"],
    ["BOLINHO DE CHUVA PRONTO", "padaria"],
    ["PÃO DE ESPECIARIAS 200G", "padaria"],
    ["TORTA SALGADA 500G", "padaria"],
    ["BISCOITO DE ARROZ 100G", "padaria"],
    ["BAGUETE RECHEADA 250G", "padaria"],

    # Bebidas
    ["REFRIGERANTE DE COLA 2L", "bebidas"],
    ["ÁGUA TÔNICA", "bebidas"],
    ["VINHO TINTO SECO", "bebidas"],
    ["SUCO DE LARANJA NATURAL 1L", "bebidas"],
    ["SUCO DE UVA INTEGRAL 1L", "bebidas"],
    ["REFRIGERANTE DE GUARANÁ 2L", "bebidas"],
    ["CERVEJA ARTESANAL 500ML", "bebidas"],
    ["CHA DE CAMOMILA 10 SAQUINHOS", "bebidas"],
    ["ENERGÉTICO 250ML", "bebidas"],
    ["ÁGUA DE COCO 1L", "bebidas"],
    ["CAFÉ SOLÚVEL 100G", "bebidas"],
    ["CHÁ VERDE 20 SAQUINHOS", "bebidas"],
    ["REFRIGERANTE SABOR LARANJA 2L", "bebidas"],
    ["LEITE DE AMÊNDOAS 1L", "bebidas"],
    ["CHÁ DE ERVA-DOCE 10 SAQUINHOS", "bebidas"],
    ["SUCO DE ABACAXI 1L", "bebidas"],
    ["REFRIGERANTE DE LIMÃO 2L", "bebidas"],
    ["ENERGÉTICO NATURAL 250ML", "bebidas"],
    ["CAFÉ MOÍDO 250G", "bebidas"],
    ["SUCO DE MANGA 1L", "bebidas"],
    ["ÁGUA AROMATIZADA 500ML", "bebidas"],
    ["SUCO DE TOMATE 340ML", "bebidas"],
    ["LICOR DE CACAU 500ML", "bebidas"],
    ["CHÁ DE HIBISCO 10 SAQUINHOS", "bebidas"],
    ["SUCO DE TOMATE 500ML", "bebidas"],
    ["LIMONADA 1L", "bebidas"],
    ["ÁGUA DE COCO 500ML", "bebidas"],
    ["CAFÉ GELADO 200ML", "bebidas"],
    ["SMOOTHIE DE FRUTAS 300ML", "bebidas"],
    ["SUCO DE MELANCIA 1L", "bebidas"],
    ["REFRIGERANTE DE LARANJA 2L", "bebidas"],
    ["SUCO DE MAÇÃ ORGÂNICO 1L", "bebidas"],
    ["CHÁ DE HORTELÃ 10 SAQUINHOS", "bebidas"],
    ["ÁGUA MINERAL COM GÁS 500ML", "bebidas"],
    ["CHÁ DE JASMIM 10 SAQUINHOS", "bebidas"],
    ["SUCO DE ACEROLA 1L", "bebidas"],
    ["CAFÉ DE CEREJA 200G", "bebidas"],
    ["SUCO DE UVA INTEGRAL 500ML", "bebidas"],
    ["ÁGUA TÔNICA FLAVORIZADA 350ML", "bebidas"],
    ["SUCO DE MELÃO 1L", "bebidas"],
    ["SUCO DE GRANADA 500ML", "bebidas"],
    ["CHA DE ERVA CIDREIRA 10 SAQUINHOS", "bebidas"],
    ["SUCO DE CAJU 1L", "bebidas"],
    ["REFRIGERANTE DIET DE FRUTAS 2L", "bebidas"],
    ["CHÁ VERDE COM LIMÃO 500ML", "bebidas"],
    ["SUCO DE PERA 1L", "bebidas"],
    ["CHÁ PRETO 10 SAQUINHOS", "bebidas"],
    ["CAFÉ MOCHA 200G", "bebidas"],
    ["CAFÉ FRAPPÉ 200ML", "bebidas"],
    ["SUCO DE AÇAÍ 500ML", "bebidas"],
    ["REFRIGERANTE DE TANGERINA 2L", "bebidas"],
    ["SUCO DE MANGABA 1L", "bebidas"],
    ["ÁGUA FLAVORIZADA DE LIMÃO 500ML", "bebidas"],
    ["CHÁ DE MENTA 10 SAQUINHOS", "bebidas"],

    # Frios
    ["PRESUNTO PARMA", "frios"],
    ["SALMÃO DEFUMADO", "frios"],
    ["QUEIJO MINAS FRESCAL", "frios"],
    ["MORTADELA 200G", "frios"],
    ["SALPICÃO 200G", "frios"],
    ["PRESUNTO DE FRANGO 200G", "frios"],
    ["QUEIJO GORGONZOLA 100G", "frios"],
    ["ATUM DEFUMADO 100G", "frios"],
    ["BLANQUET DE PERU 200G", "frios"],
    ["ROLO DE QUEIJO COM PRESUNTO", "frios"],
    ["SALSICHA VIENA 300G", "frios"],
    ["PRESUNTO COZIDO 200G", "frios"],
    ["COPA FATIADA 100G", "frios"],
    ["SALSICHÃO 200G", "frios"],
    ["PEITO DE PERU DEFUMADO 200G", "frios"],
    ["SALAMINHO 100G", "frios"],
    ["COTTAGE 150G", "frios"],
    ["BLANQUET DE PERU 150G", "frios"],
    ["SALSICHA TIPO VIENA 300G", "frios"],
    ["PRESUNTO CRU 100G", "frios"],
    ["BLANQUETE DE PERU 200G", "frios"],
    ["SALAME HÚNGARO 100G", "frios"],
    ["MORTADELA COM PIMENTA 200G", "frios"],
    ["PEITO DE FRANGO DEFUMADO 200G", "frios"],
    ["MUSSARELA DE BÚFALA 200G", "frios"],
    ["PRESUNTO COZIDO FATIADO 200G", "frios"],
    ["SALAME ITALIANO 200G", "frios"],
    ["PEITO DE PERU LIGHT 200G", "frios"],
    ["SALSICHA BOCK 200G", "frios"],
    ["PRESUNTO PARMA 100G", "frios"],
    ["SALMÃO DEFUMADO 100G", "frios"],
    ["LINGUIÇA BLUMENAU 200G", "frios"],
    ["PATE DE ATUM 150G", "frios"],
    ["SALAME HÚNGARO 200G", "frios"],
    ["MORTADELA ITALIANA 200G", "frios"],

    # Limpeza
    ["AMACIANTE DE ROUPAS", "limpeza"],
    ["Detergente Líquido", "limpeza"],
    ["ESPONJA DE LÃ DE AÇO", "limpeza"],
    ["LIMPADOR MULTIUSO", "limpeza"],
    ["DETERGENTE EM PÓ 1KG", "limpeza"],
    ["LIMPADOR DE VIDROS 500ML", "limpeza"],
    ["DESINFETANTE LAVANDA 1L", "limpeza"],
    ["AMACIANTE CONCENTRADO 500ML", "limpeza"],
    ["SABÃO EM BARRA 5 UNIDADES", "limpeza"],
    ["LIMPADOR DE PISO 1L", "limpeza"],
    ["SACO DE LIXO 30L 20 UNIDADES", "limpeza"],
    ["ALVEJANTE SEM CLORO 500ML", "limpeza"],
    ["ESPONJA MULTIUSO 3 UNIDADES", "limpeza"],
    ["CERA PARA PISO 750ML", "limpeza"],
    ["DESINFETANTE PINHO 1L", "limpeza"],
    ["LIMPADOR MULTIUSO CÍTRICO 500ML", "limpeza"],
    ["SABÃO EM PÓ BIODEGRADÁVEL 1KG", "limpeza"],
    ["DETERGENTE BIODEGRADÁVEL 500ML", "limpeza"],
    ["LIMPA VIDROS 500ML", "limpeza"],
    ["SABÃO LÍQUIDO PARA ROUPAS 2L", "limpeza"],
    ["AMACIANTE DE ROUPAS AROMATIZADO 2L", "limpeza"],
    ["SAPONÁCEO CREMOSO 250ML", "limpeza"],
    ["LIMPADOR DE INOX 200ML", "limpeza"],
    ["LIMPADOR DE COURO 100ML", "limpeza"],
    ["DETERGENTE LÍQUIDO NEUTRO 500ML", "limpeza"],
    ["LIMPADOR DE MÓVEIS 200ML", "limpeza"],
    ["AMACIANTE DE ROUPAS AROMATIZADO 1L", "limpeza"],
    ["ALVEJANTE PARA ROUPAS COLORIDAS 1L", "limpeza"],
    ["LIMPADOR DE ALUMÍNIO 500ML", "limpeza"],
    ["SABÃO EM PASTA 200G", "limpeza"],
    ["ÁGUA SANITÁRIA 2L", "limpeza"],
    ["LIMPADOR DE VIDROS SPRAY 500ML", "limpeza"],
    ["LIMPADOR DE PISOS ANTIBACTERIANO 1L", "limpeza"],
    ["DESODORIZANTE PARA AMBIENTES 300ML", "limpeza"],
    ["BALDE PLÁSTICO 10L", "limpeza"],
    ["CERA LÍQUIDA PARA PISO 750ML", "limpeza"],
    ["DESINFETANTE PARA BANHEIRO 500ML", "limpeza"],
    ["DETERGENTE DESINFETANTE 1L", "limpeza"],
    ["ESPONJA MÁGICA", "limpeza"],
    ["LIMPADOR DE TAPETE EM PÓ 500G", "limpeza"],
    ["LUVA DE LIMPEZA", "limpeza"],
    ["MOP GIRATÓRIO", "limpeza"],
    ["PANO DE MICROFIBRA", "limpeza"],
    ["RODO DE PIA", "limpeza"],
    ["SABÃO EM PÓ PARA ROUPAS DELICADAS 1KG", "limpeza"],
    ["SACO PARA LIXO RECICLÁVEL 50L", "limpeza"],
    ["SOLUÇÃO LIMPA VIDROS 500ML", "limpeza"],
    ["TIRA MANCHAS EM PÓ 500G", "limpeza"],
    ["VASSOURA DE CERDAS MACIAS", "limpeza"],
    ["ÁCIDO MURIÁTICO 1L", "limpeza"],
    ["DESENTUPIDOR QUÍMICO 300G", "limpeza"],
    ["LIMPADOR DE INOX SPRAY 250ML", "limpeza"],
    ["REMOVEDOR DE CERAS 1L", "limpeza"],
    ["ESPONJA DE AÇO INOXIDÁVEL", "limpeza"],

    # Horti-Fruti
    ["MAÇÃ FUJI", "horti-fruti"],
    ["TOMATE CAQUI KG", "horti-fruti"],
    ["ABACAXI HAWAI", "horti-fruti"],
    ["CEBOLA ROXA KG", "horti-fruti"],
    ["ABÓBORA MORANGA KG", "horti-fruti"],
    ["MANGA TOMMY KG", "horti-fruti"],
    ["PIMENTÃO VERDE KG", "horti-fruti"],
    ["MORANGO CAIXA", "horti-fruti"],
    ["ALFACE AMERICANA UNIDADE", "horti-fruti"],
    ["CENOURA KG", "horti-fruti"],
    ["BERINJELA KG", "horti-fruti"],
    ["COUVE MINEIRA MAÇO", "horti-fruti"],
    ["BETERRABA KG", "horti-fruti"],
    ["PEPINO JAPONÊS KG", "horti-fruti"],
    ["BROCOLIS NINJA UNIDADE", "horti-fruti"],
    ["PÊRA PORTUGUESA KG", "horti-fruti"],
    ["BATATA DOCE KG", "horti-fruti"],
    ["ESPINAFRE MAÇO", "horti-fruti"],
    ["RÚCULA MAÇO", "horti-fruti"],
    ["CHUCHU KG", "horti-fruti"],
    ["ASPARGO VERDE BANDEJA", "horti-fruti"],
    ["MELÃO AMARELO KG", "horti-fruti"],
    ["FIGO FRESCO BANDEJA", "horti-fruti"],
    ["FRAMBOESA CAIXA", "horti-fruti"],
    ["KIWI KG", "horti-fruti"],
    ["JILÓ KG", "horti-fruti"],
    ["MELANCIA KG", "horti-fruti"],
    ["QUIABO KG", "horti-fruti"],
    ["NABO KG", "horti-fruti"],
    ["COGUMELO SHITAKE BANDEJA", "horti-fruti"],
    ["MAMÃO PAPAIA KG", "horti-fruti"],
    ["GOIABA VERMELHA KG", "horti-fruti"],
    ["JACA KG", "horti-fruti"],
    ["GRÃO DE BICO KG", "horti-fruti"],
    ["VAGEM KG", "horti-fruti"],
    ["UVA THOMPSON KG", "horti-fruti"],
    ["PÊSSEGO NACIONAL KG", "horti-fruti"],
    ["NECTARINA KG", "horti-fruti"],
    ["AMEIXA FRESCA KG", "horti-fruti"],
    ["MIRTILO (BLUEBERRY) CAIXA", "horti-fruti"],
    ["FIGO TURCO KG", "horti-fruti"],
    ["AMORA FRESCA CAIXA", "horti-fruti"],
    ["CACAU FRUTO KG", "horti-fruti"],
    ["ROMÃ KG", "horti-fruti"],
    ["FRAMBOESA FRESCA CAIXA", "horti-fruti"],
    ["KIWI VERDE KG", "horti-fruti"],
    ["LICHIA KG", "horti-fruti"],
    ["TAMARINDO KG", "horti-fruti"],
    ["CACAU EM PÓ 100G", "horti-fruti"],
    ["CASTANHA DE CAJU 100G", "horti-fruti"],
    ["NOZES 100G", "horti-fruti"],
    ["AMÊNDOA 100G", "horti-fruti"],
    ["CASTANHA DO PARÁ 100G", "horti-fruti"],
    ["AVELÃ 100G", "horti-fruti"],
    ["PISTACHE 100G", "horti-fruti"],

    # Eletrônicos
    ["CARREGADOR PORTÁTIL USB", "eletrônicos"],
    ["MOUSE SEM FIO", "eletrônicos"],
    ["CABO HDMI 2M", "eletrônicos"],
    ["PILHAS AAA 4 UNIDADES", "eletrônicos"],
    ["LANTERNA DE LED", "eletrônicos"],
    ["RÁDIO PORTÁTIL", "eletrônicos"],
    ["FONE DE OUVIDO COM FIO", "eletrônicos"],
    ["CONTROLE REMOTO UNIVERSAL", "eletrônicos"],
    ["PEN DRIVE 16GB", "eletrônicos"],
    ["CARREGADOR VEICULAR USB", "eletrônicos"],
    ["TELEFONE SEM FIO", "eletrônicos"],
    ["ADAPTADOR USB", "eletrônicos"],
    ["CARREGADOR PORTÁTIL SOLAR", "eletrônicos"],
    ["TECLADO MECÂNICO", "eletrônicos"],
    ["MOUSEPAD GAMER", "eletrônicos"],
    ["CÂMERA DIGITAL COMPACTA", "eletrônicos"],
    ["FONE DE OUVIDO SEM FIO", "eletrônicos"],
    ["SMARTWATCH", "eletrônicos"],
    ["ANTENA DIGITAL PARA TV", "eletrônicos"],
    ["CABO DE REDE 5M", "eletrônicos"],
    ["PILHAS D", "eletrônicos"],
    ["CABO USB MICRO 1M", "eletrônicos"],
    ["LÂMPADA DE HALOGÊNIO 50W", "eletrônicos"],
    ["EXTENSÃO ELÉTRICA 5M", "eletrônicos"],
    ["CABO AUXILIAR 3.5MM", "eletrônicos"],
    ["CARREGADOR PARA CARRO", "eletrônicos"],
    ["FONE DE OUVIDO INFANTIL", "eletrônicos"],
    ["CÂMERA DE AÇÃO", "eletrônicos"],
    ["TECLADO BLUETOOTH", "eletrônicos"],
    ["ADAPTADOR HDMI PARA VGA", "eletrônicos"],
    ["RÁDIO AM/FM PORTÁTIL", "eletrônicos"],
    ["VENTILADOR DE MESA", "eletrônicos"],
    ["BATERIA EXTERNA PORTÁTIL", "eletrônicos"],
    ["WEB CAM HD", "eletrônicos"],
    ["RELÓGIO DESPERTADOR DIGITAL", "eletrônicos"],
    ["CABO DE FORÇA 3 PINOS", "eletrônicos"],
    ["CARREGADOR SEM FIO", "eletrônicos"],
    ["TELEFONE COM FIO", "eletrônicos"],
    ["MOUSE ÓPTICO COM FIO", "eletrônicos"],
    ["SUPORTE PARA MONITOR", "eletrônicos"],
    ["CAIXA DE SOM USB", "eletrônicos"],
    ["MINI DRONE", "eletrônicos"],
    ["ROTEADOR WIRELESS", "eletrônicos"],
    ["CONTROLE PARA VIDEOGAME", "eletrônicos"],
    ["CÂMERA FILMADORA PORTÁTIL", "eletrônicos"],
    ["FONE DE OUVIDO NOISE CANCELING", "eletrônicos"],
    ["TECLADO GAMER RGB", "eletrônicos"],
    ["MONITOR LED 24 POLEGADAS", "eletrônicos"],
    ["IMPRESSORA MULTIFUNCIONAL", "eletrônicos"],
    ["TABLET 8 POLEGADAS", "eletrônicos"],
]

validation_products = [
    # Doces
    ["GELEIA DE AMORA CASEIRA", "doces"],
    ["Chocolate Branco 100g", "doces"],
    ["Bala de Goma 100g", "doces"],
    ["Doce de Leite Pote 400g", "doces"],
    ["ALFAJOR 100G", "doces"],
    ["BOMBOM DE CEREJA 200G", "doces"],
    ["BARRA DE CHOCOLATE BRANCO COM AMENDOIM", "doces"],
    ["BALA DE BANANA 100G", "doces"],
    ["BISCOITO DE CHOCOLATE 150G", "doces"],
    ["GELÉIA DE UVA 200G", "doces"],
    ["BOLO DE LARANJA 300G", "doces"],
    ["WAFER DE BAUNILHA 100G", "doces"],
    ["CHOCOLATE EM PÓ 200G", "doces"],
    ["BISCOITO AMANTEIGADO DE LIMÃO 100G", "doces"],

    # Laticínios
    ["REQUEIJÃO CREMOSO 200G", "laticínios"],
    ["Queijo Prato KG", "laticínios"],
    ["Manteiga com Sal 200g", "laticínios"],
    ["YOGURTE DE MORANGO 170G", "laticínios"],
    ["LEITE AROMATIZADO CHOCOLATE 200ML", "laticínios"],
    ["QUEIJO EMMENTAL 100G", "laticínios"],
    ["CREME DE LEITE LIGHT 200G", "laticínios"],
    ["MANTEIGA SEM SAL 200G", "laticínios"],
    ["IOGURTE DE FRUTAS VERMELHAS 170G", "laticínios"],
    ["LEITE SEMIDESNATADO 1L", "laticínios"],
    ["QUEIJO MINAS FRESCAL 200G", "laticínios"],
    ["IOGURTE LÍQUIDO DE PÊSSEGO 200ML", "laticínios"],

    # Carnes
    ["FILÉ DE PEIXE TILÁPIA CONGELADO", "carnes"],
    ["Carne Moída de Primeira KG", "carnes"],
    ["Picanha Bovina KG", "carnes"],
    ["Asa de Frango KG", "carnes"],
    ["LINGUIÇA DE FRANGO 500G", "carnes"],
    ["COSTELA BOVINA DEFUMADA KG", "carnes"],
    ["HAMBÚRGUER DE FRANGO 4 UNIDADES", "carnes"],
    ["ALMONDEGAS DE CARNE BOVINA 300G", "carnes"],
    ["COSTELINHA SUÍNA AO MOLHO BARBECUE", "carnes"],
    ["FRANGO INTEIRO CONGELADO", "carnes"],
    ["CARNE DE PORCO PARA CHURRASCO KG", "carnes"],
    ["CARNE DE SOL 500G", "carnes"],
    ["FRANGO DESFIADO CONGELADO 500G", "carnes"],
    ["PERU INTEIRO CONGELADO", "carnes"],

    # Grãos e Cereais
    ["GRANOLA TRADICIONAL 300G", "grãos e cereais"],
    ["Feijão Preto 1kg", "grãos e cereais"],
    ["Arroz Parboilizado 5kg", "grãos e cereais"],
    ["Quinoa em Grãos 500g", "grãos e cereais"],
    ["CEREAL MATINAL DE CHOCOLATE", "grãos e cereais"],
    ["SEMENTE DE GIRASSOL 100G", "grãos e cereais"],
    ["FARINHA DE ARROZ 1KG", "grãos e cereais"],
    ["MILHO PARA PIPOCA 500G", "grãos e cereais"],
    ["FARINHA DE AMÊNDOAS 200G", "grãos e cereais"],
    ["GRÃO DE BICO 500G", "grãos e cereais"],
    ["FLOCOS DE MILHO COM AÇÚCAR", "grãos e cereais"],
    ["ARROZ NEGRO 1KG", "grãos e cereais"],
    ["FARINHA DE LINHAÇA 500G", "grãos e cereais"],
    ["FARINHA DE BANANA VERDE 200G", "grãos e cereais"],

    # Não Perecível
    # ["AZEITONAS VERDES RECHEADAS", "não perecível"],
    # ["COGUMELOS EM CONSERVA", "não perecível"],
    # ["Conserva de Azeitonas 200g", "não perecível"],
    # ["Milho Verde em Conserva Lata", "não perecível"],
    # ["Sopa Instantânea 65g", "não perecível"],
    # ["MOLHO DE ALHO 200ML", "não perecível"],
    # ["AZEITE DE DENDÊ 200ML", "não perecível"],
    # ["VINAGRE BALSÂMICO 250ML", "não perecível"],
    # ["AZEITONAS PRETAS 200G", "não perecível"],
    # ["MOLHO DE SOJA LIGHT 150ML", "não perecível"],
    # ["PALMITO PUPUNHA EM CONSERVA", "não perecível"],
    # ["PICKLES EM CONSERVA 200G", "não perecível"],
    # ["TOMATE SECO EM ÓLEO 200G", "não perecível"],
    # ["CONDIMENTO PARA PIZZA", "não perecível"],
    # ["MASSA DE TOMATE 140G", "não perecível"],

    # Padaria
    ["PÃO DE QUEIJO CONGELADO", "padaria"],
    ["Baguete Francesa Unidade", "padaria"],
    ["Croissant Manteiga Unidade", "padaria"],
    ["Bolo de Cenoura com Cobertura de Chocolate", "padaria"],
    ["PÃO DE ALHO PARA CHURRASCO", "padaria"],
    ["TORTA DE MAÇÃ 300G", "padaria"],
    ["PÃO CIABATTA 200G", "padaria"],
    ["PÃO DE CENOURA 200G", "padaria"],
    ["ROSCA DOCE COM COCO 200G", "padaria"],
    ["BISCOITO DE QUEIJO 100G", "padaria"],
    ["PÃO DE AVEIA 200G", "padaria"],
    ["DONUTS COM COBERTURA DE CHOCOLATE", "padaria"],
    ["CHEESECAKE 300G", "padaria"],
    ["BISCOITO CREAM CRACKER 200G", "padaria"],

    # Bebidas
    ["ÁGUA MINERAL SEM GÁS 500ML", "bebidas"],
    ["Suco de Maçã Integral 1L", "bebidas"],
    ["Cerveja Pilsen 350ml", "bebidas"],
    ["Vinho Branco Seco 750ml", "bebidas"],
    ["SUCO DE GOIABA 1L", "bebidas"],
    ["CHÁ GELADO DE PÊSSEGO 500ML", "bebidas"],
    ["ISOTÔNICO SABOR LIMÃO 500ML", "bebidas"],
    ["SUCO DETOX VERDE 300ML", "bebidas"],
    ["CHÁ DE GENGIBRE COM LIMÃO", "bebidas"],
    ["CAPPUCCINO EM PÓ 200G", "bebidas"],
    ["ÁGUA TÔNICA DIET", "bebidas"],
    ["SUCO DE BETERRABA COM LARANJA 1L", "bebidas"],
    ["BEBIDA ENERGÉTICA SEM AÇÚCAR 250ML", "bebidas"],
    ["CAFÉ EXPRESSO 50ML", "bebidas"],

    # Higiene
    ["PASTA DE DENTE MENTA REFRESCANTE", "higiene"],
    ["Papel Toalha 2 Rolos", "higiene"],
    ["Lenços Umedecidos 50 Unidades", "higiene"],
    ["Sabonete Líquido Antibacteriano 250ml", "higiene"],
    ["SABONETE ANTIBACTERIANO 90G", "higiene"],
    ["ÁGUA MICELAR 200ML", "higiene"],
    ["SHAMPOO PARA CABELOS CACHEADOS 200ML", "higiene"],
    ["DESODORANTE HIPOALERGÊNICO 50ML", "higiene"],
    ["CONDICIONADOR PARA CABELOS COLORIDOS 200ML", "higiene"],
    ["SABONETE ESFOLIANTE 100G", "higiene"],
    ["CREME DENTAL CLAREADOR 90G", "higiene"],
    ["LENÇOS DEMAQUILANTES 25 UNIDADES", "higiene"],
    ["LOÇÃO PÓS-BARBA 100ML", "higiene"],
    ["ANTISSÉPTICO BUCAL SEM ÁLCOOL 250ML", "higiene"],

    # Horti-Fruti
    ["Banana Prata KG", "horti-fruti"],
    ["Limão Tahiti KG", "horti-fruti"],
    ["Laranja Pera KG", "horti-fruti"],
    ["CHAMPINHON FRESCO BANDEJA", "horti-fruti"],
    ["ERVAS FINAS MAÇO", "horti-fruti"],
    ["PEPINO AZEITONA KG", "horti-fruti"],
    ["BETERRABA BABY KG", "horti-fruti"],
    ["FIGO DA ÍNDIA KG", "horti-fruti"],
    ["BROTO DE FEIJÃO 200G", "horti-fruti"],
    ["MARACUJÁ KG", "horti-fruti"],
    ["ABOBORA CABOTIÁ KG", "horti-fruti"],
    ["PITAYA KG", "horti-fruti"],

    # Limpeza
    ["Desinfetante Eucalipto 2L", "limpeza"],
    ["Água Sanitária 1L", "limpeza"],
    ["Saco de Lixo 50L Rolo", "limpeza"],
    ["PANO DE CHÃO", "limpeza"],
    ["LIMPADOR DESINFETANTE 500ML", "limpeza"],
    ["SABÃO EM BARRA PARA ROUPAS 200G", "limpeza"],
    ["LIMPADOR DE MADEIRA 500ML", "limpeza"],
    ["ÁLCOOL 70% 1L", "limpeza"],
    ["LIMPADOR DE AZULEJOS 500ML", "limpeza"],
    ["SABÃO EM PÓ PARA ROUPAS COLORIDAS 1KG", "limpeza"],
    ["DETERGENTE EM GEL 500ML", "limpeza"],
    ["ESPONJA DUPLA FACE", "limpeza"],
    ["LIMPADOR DE TAPETES 500ML", "limpeza"],

    # Eletrônicos
    ["Lâmpada LED 9W", "eletrônicos"],
    ["Carregador de Pilhas com 4 AA", "eletrônicos"],
    ["Fone de Ouvido Bluetooth", "eletrônicos"],
    ["CARREGADOR DE CELULAR", "eletrônicos"],
    ["PILHAS RECARREGÁVEIS AA", "eletrônicos"],
    ["CABO USB TIPO C 1M", "eletrônicos"],
    ["FONE DE OUVIDO COM MICROFONE", "eletrônicos"],
    ["CÂMERA DE SEGURANÇA WIFI", "eletrônicos"],
    ["EXTENSÃO ELÉTRICA 3M", "eletrônicos"],
    ["ADAPTADOR DE TOMADA UNIVERSAL", "eletrônicos"],
    ["CAIXA DE SOM BLUETOOTH", "eletrônicos"],
    ["ANTENA DE TV INTERNA", "eletrônicos"],
    ["LÂMPADA INTELIGENTE LED RGB", "eletrônicos"],

    # Congelados
    ["Hambúrguer Bovino Pacote 6 Unidades", "congelados"],
    ["Polpa de Fruta Congelada 100g", "congelados"],
    ["COXINHA DE FRANGO CONGELADA", "congelados"],
    ["PEIXE ESPADA FILE CONGELADO", "congelados"],
    ["VEGETAIS MISTOS CONGELADOS", "congelados"],
    ["PIZZA DE CALABRESA CONGELADA", "congelados"],
    ["NUGGETS DE FRANGO CONGELADO", "congelados"],
    ["EMPANADO DE CAMARÃO CONGELADO", "congelados"],
    ["ESCONDIDINHO DE CARNE CONGELADO", "congelados"],
    ["SALGADINHOS VARIADOS CONGELADOS", "congelados"],
    ["TORTA DE PALMITO CONGELADA", "congelados"],

    # Frios
    ["Peito de Peru Defumado 200g", "frios"],
    ["Salame Italiano Fatias 100g", "frios"],
    ["Provolone Desidratado 100g", "frios"],
    ["SALAME TIPO ITALIANO 100G", "frios"],
    ["PROSCIUTTO FATIADO 100G", "frios"],
    ["PRESUNTO DE PARMA 100G", "frios"],
    ["PEITO DE PERU DEFUMADO FATIADO 200G", "frios"],
    ["MORTADELA COM AZEITONAS 200G", "frios"],
    ["PASTA DE ATUM 150G", "frios"],
]

In [25]:
categorias = [
    "horti-fruti",
    "laticínios",
    "doces",
    "carnes",
    "grãos e cereais",
    # "não perecível",
    "padaria",
    "bebidas",
    "higiene",
    "limpeza",
    "eletrônicos",
    "congelados",
    "frios",
]

# Dataset Validation

In [26]:
full_dataset = produtos + validation_products
# Validação
verificar_categorias(full_dataset, categorias)
verificar_duplicatas(full_dataset)
verificar_itens_nao_categorizados(full_dataset)
validar_rotulos_de_classe(full_dataset, categorias)

Todas as categorias são válidas.
Nenhuma duplicata encontrada.
Todos os itens estão categorizados.
Todos os rótulos de classe são válidos.


# Label Encoding

In [27]:
# Função para codificar os rótulos do dataset
def codificar_rotulos(dataset, encoder):
    # Codifica os rótulos usando o encoder
    for produto in dataset:
        produto[1] = encoder.transform([produto[1]])[0]

In [28]:
from sklearn.preprocessing import LabelEncoder

# Inicialização do LabelEncoder e treinamento com todas as categorias
encoder = LabelEncoder()
encoder.fit(categorias)

# Codificar os rótulos nos datasets de treinamento e validação
codificar_rotulos(produtos, encoder)
codificar_rotulos(validation_products, encoder)

# Google Drive Daemon

In [29]:
import os
import shutil
from google.colab import drive

drive.mount('/content/drive')

def copy_folder_to_drive(folder_name, target_drive_folder='/content/drive/MyDrive/ML/checkpoints'):
    # Monta o Google Drive

    # Define o caminho completo da pasta de origem e destino
    source_folder = os.path.join('./results', folder_name)
    destination_folder = os.path.join(target_drive_folder, folder_name)

    # Verifica se a pasta de origem existe
    if not os.path.exists(source_folder):
        print(f"A pasta '{source_folder}' não existe.")
        return

    # Verifica se a pasta de destino já existe no Drive
    if os.path.exists(destination_folder):
        print(f"A pasta '{destination_folder}' já existe no Drive.")
        return

    # Copia a pasta
    shutil.copytree(source_folder, destination_folder)
    print(f"Pasta '{folder_name}' copiada com sucesso para '{destination_folder}'")

def fetch_checkpoint(checkpoint_folder):
  checkpoint_path = f"/content/drive/MyDrive/ML/checkpoints/{checkpoint_folder}"

  destination_path = f"./results/{checkpoint_folder}"

  shutil.copytree(checkpoint_path, destination_path)

  print(f'Pasta do checkpoint copiada para: {destination_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
def absolute_dir(model_name):
    return os.path.join('/content/results', model_name)

# Fetch last checkpoint

In [31]:
enable_fetch = False
if enable_fetch:
  fetch_checkpoint("checkpoint-350")
  __CURRENT_TRAINING_MODEL__ = get_last_trained_model_name(__RESULTS_PATH__, __TRAINING_MODEL__)
  log(f"Novo modelo em treinamento: {__CURRENT_TRAINING_MODEL__}", green_color)
else:
  log(f"Fetch disabled, skipping")

Fetch disabled, skipping


# Training definitions

In [32]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_metric
from collections import Counter

In [47]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk

# Certifique-se de baixar os pacotes necessários
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

def remove_non_alphanumeric(text):
    """
    Remove caracteres não alfanuméricos de um texto.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

def to_lowercase(text):
    """
    Converte todo o texto para minúsculas.
    """
    return text.lower()

def remove_stopwords(text, language='english'):
    """
    Remove stopwords do texto.
    """
    stop_words = set(stopwords.words(language))
    return ' '.join([word for word in text.split() if word not in stop_words])

def stem_text(text, language='english'):
    """
    Aplica stemming ao texto.
    """
    stemmer = SnowballStemmer(language)
    return ' '.join([stemmer.stem(word) for word in text.split()])

def lemmatize_text(text):
    """
    Aplica lematização ao texto.
    """
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# Função de pré-processamento que combina todas as etapas acima
def preprocess_text(text, remove_stopwords_flag=True, use_stemming=False):
    text = remove_non_alphanumeric(text)
    text = to_lowercase(text)
    if remove_stopwords_flag:
        text = remove_stopwords(text)
    if use_stemming:
        text = stem_text(text)
    else:
        text = lemmatize_text(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
def most_likely(model, tokenizer, texto, labels):
    # Obtenha a previsão mais provável do modelo treinado
    prediction = predict_with_model(model, tokenizer, [texto])
    melhor_categoria = prediction[0]
    return melhor_categoria

def iterate_list(model, tokenizer, list_items, categories, max_iterations=None):
    match_count = 0
    iterations = range(len(list_items)) if max_iterations is None else range(min(max_iterations, len(list_items)))

    for index in iterations:
        item, expected_category = list_items[index]
        print(f"Iteração Número {index + 1}\n")
        print(f"Avaliando entrada: {item}")

        # Obtenha a previsão mais provável do modelo treinado
        predicted_category = most_likely(model, tokenizer, item, categories)
        print(f"Categoria prevista: {predicted_category}")

        # Verifica se a previsão está correta
        if predicted_category == expected_category:
            match_count += 1

    total_iterations = len(iterations)
    efficiency_rate = (match_count / total_iterations) * 100
    print(f"Total de correspondências: {match_count}, Taxa de aproveitamento: {efficiency_rate:.2f}%")

    return match_count

In [35]:
# Função para fazer predições com o modelo treinado
def predict_with_model(model, tokenizer, texts, ids_to_label):
    # Predições
    outputs = model(**texts)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return [ids_to_label[id.item()] for id in predictions]

In [58]:
def evaluate_predictions(model, tokenizer, products):
    correct_predictions = 0
    total_predictions = len(products)
    device = next(model.parameters()).device  # Pega o dispositivo do modelo

    for product, true_label in products:
        # Certifique-se de que 'product' é uma string
        # Se 'product' já for uma string, você pode comentar a linha abaixo
        product_text = preprocess_text(product)  # Supondo que preprocess_text retorna uma string

        # Processamento do texto para o formato aceito pelo modelo
        assert isinstance(product_text, str), "product_text deve ser uma string"
        inputs = tokenizer(product_text, truncation=True, padding=True, max_length=128, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}  # Movendo para o dispositivo correto

        # A função `predict_with_model` espera uma lista de textos
        predicted_label = predict_with_model(model, tokenizer, inputs, ids_to_label)[0]
        is_correct = predicted_label == true_label
        correct_predictions += is_correct

        decoded_true_label = encoder.inverse_transform([true_label])[0]
        decoded_predicted_label = encoder.inverse_transform([predicted_label])[0]

        # Imprimindo resultados
        log(f"Produto: {product_text}", blue_color)
        log(f"Categoria Verdadeira: {decoded_true_label}", reset_color)
        log(f"Categoria Prevista: {decoded_predicted_label}", reset_color)
        log(f"Previsão Correta: {is_correct}\n", (green_color if is_correct else red_color))

    # Cálculo da precisão
    accuracy = (correct_predictions / total_predictions) * 100
    __LAST_MODEL_ACCURACY__ = f"{accuracy:.2f}%"
    log(f"Acurácia do Modelo Atual: {__LAST_MODEL_ACCURACY__}", red_color)


In [37]:
# Conversão dos dados tokenizados para um formato de Dataset do PyTorch
class ProductsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Model Loader

In [38]:
from torch.cuda import is_available
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Verificando se a GPU está disponível
device = 'cuda' if is_available() else 'cpu'
print(f"Using {device}")

# Separando os dados em textos e labels
textos = [produto[0] for produto in produtos]
labels = [produto[1] for produto in produtos]

# Caminho do modelo
__CURRENT_TRAINING_MODEL__ = get_last_trained_model_name(__RESULTS_PATH__, __TRAINING_MODEL__)
absolute_model_path = __TRAINING_MODEL__ if __CURRENT_TRAINING_MODEL__ == __TRAINING_MODEL__ else absolute_dir(__CURRENT_TRAINING_MODEL__)
print("Caminho absoluto do modelo:", absolute_model_path)

# Carregamento do tokenizer
tokenizer = AutoTokenizer.from_pretrained(__TRAINING_MODEL__)

# Aplicando o pré-processamento aos textos
processed_texts = [preprocess_text(text) for text in textos]

# Tokenização dos dados
encodings = tokenizer(processed_texts, truncation=True, padding=True, max_length=128)

# Criando o dataset
dataset = ProductsDataset(encodings, labels)

# Criação do DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=False)

# Carregamento dos batches para o dispositivo
for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

# Divisão dos dados em conjuntos de treinamento e validação
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Carregamento do modelo
model = AutoModelForSequenceClassification.from_pretrained(absolute_model_path, num_labels=len(encoder.classes_), ignore_mismatched_sizes=True)
model = model.to(device)

Using cpu
Caminho absoluto do modelo: /content/results/checkpoint-350


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2', '3': 'LABEL_3', '4': 'LABEL_4', '5': 'LABEL_5', '6': 'LABEL_6', '7': 'LABEL_7', '8': 'LABEL_8', '9': 'LABEL_9', '10': 'LABEL_10', '11': 'LABEL_11', '12': 'LABEL_12'}. The number of labels wil be overwritten to 13.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at /content/results/checkpoint-350 and are newly initialized because the shapes did not match:
- classification_head.out_proj.bias: found shape torch.Size([13]) in the checkpoint and torch.Size([12]) in the model instantiated
- classification_head.out_proj.weight: found shape torch.Size([13, 1024]) in the checkpoint and torch.Size([12, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Model Training

In [39]:
# Definição dos argumentos de treinamento
training_args = TrainingArguments(
    num_train_epochs=__EPOCHS__,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    output_dir=__RESULTS_PATH__,
    logging_dir='./logs',
    logging_strategy="steps",  # Pode ser "epoch", "steps" ou "no"
    logging_steps=2,
)

# Definição do Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Treinamento do modelo
trainer.train()

# Avaliação do modelo
trainer.evaluate()


Epoch,Training Loss,Validation Loss
1,0.170500,0.115705
2,0.013800,0.329191
3,0.016100,0.235596
4,0.003400,0.240199
5,0.002100,0.271538


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight'].


{'eval_loss': 0.11570513993501663,
 'eval_runtime': 27.4521,
 'eval_samples_per_second': 4.517,
 'eval_steps_per_second': 0.073,
 'epoch': 5.0}

# Save checkpoint

In [43]:
# Salva o modelo no drive.
copy_folder_to_drive(__CURRENT_TRAINING_MODEL__)

Pasta 'checkpoint-155' copiada com sucesso para '/content/drive/MyDrive/ML/checkpoints/checkpoint-155'


# Análise do Modelo

In [57]:
# Testa modelo final.
evaluate_predictions(model, tokenizer, validation_products)

# Atualiza modelo atual.
__CURRENT_TRAINING_MODEL__ = get_last_trained_model_name(__RESULTS_PATH__, __TRAINING_MODEL__)
log(f"Saved model: {__CURRENT_TRAINING_MODEL__}", blue_color)


Produto: geleia de amora caseira
Categoria Verdadeira: doces
Categoria Prevista: horti-fruti
Previsão Correta: False

Acurácia do Modelo Atual: 0.00%
Saved model: checkpoint-155
